## Imports and path to the file

In [1]:
from imports import pymysql, json
from config import host, user_name, db_name, password

path = "/Users/alexraudvee/Desktop/TU:e/data_challenge_1/airlines-1558527599826.json"

## Create the list with tweet objects from the file

In [2]:
lst_main = []

with open(path) as file:
    for line in file:
        lst_main.append(json.loads(line.strip()))


## Get the user data

In [5]:
for tweet_object in lst_main[0:3]:
    print(tweet_object['user'])

{'id': 393374091, 'id_str': '393374091', 'name': 'EP Turismo', 'screen_name': 'EPturismo', 'location': 'Madrid, Spain', 'url': 'http://www.europapress.es/turismo/', 'description': 'Todo sobre política turística, hoteles, touroperadores, transporte... así como reportajes sobre destinos turísticos y viajes de gran atractivo.', 'translator_type': 'none', 'protected': False, 'verified': True, 'followers_count': 44323, 'friends_count': 845, 'listed_count': 1152, 'favourites_count': 394, 'statuses_count': 73224, 'created_at': 'Tue Oct 18 12:55:25 +0000 2011', 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': 'E4E8EF', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_tile': False, 'profile_link_color': '4A913C', 'profile_sidebar_border_color':

In [6]:
users_fill_query = """
    INSERT INTO 'users' ('user_id', 'user_name', 'user_screen_name', 'followers_count', 'friends_count') VALUES (%s, %s, %s, %s, %s)
"""

users_fill_values = []

for tweet_object in lst_main[0:3]:

    user_tpl = (tweet_object['user']['id'],  # user_id
                 tweet_object['user']['name'], # user_name
                 tweet_object['user']['screen_name'],  # user_screen_name
                 tweet_object['user']['followers_count'],  # followers_count
                 tweet_object['user']['friends_count']) # friends_count
    
    users_fill_values.append(user_tpl)
    
print(users_fill_values)

[(393374091, 'EP Turismo', 'EPturismo', 44323, 845), (3420691215, 'hounddog', 'hounddog71', 1260, 1468), (394376606, 'Alexander Viol', 'AlexanderViol', 92, 215)]


## Get the replies tweet objects

In [7]:
print(lst_main[0]['entities']['hashtags'])

[]


In [8]:
replies_fill_query = """
    INSERT INTO 'replies' ( 'tweet_id', 
                            'in_reply_to_status_id', 
                            'in_reply_to_user_id', 
                            'user_id', 
                            'full_text', 
                            'timestamp_ms', 
                            'lang', 
                            'hashtags',
                            'symbols',
                            'user_mentions',
                            'quote_count',
                            'reply_count',
                            'favorite_count',
                            'retweeted_count',
                            'possibly_sensetive' )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

replies_fill_values = []

for tweet_object in lst_main:

    try:
        if tweet_object['in_reply_to_status_id'] != None:
            try: 
                if tweet_object['truncated'] == True:
                    full_text = tweet_object['extended_tweet']['full_text']

                if tweet_object['truncated'] == False:
                    full_text = tweet_object['text']
            except:
                full_text = None

            user_mentions = tweet_object['entities']['user_mentions']

            try:
                user_mentions_id = []
                for user in user_mentions:
                    user_mentions_id.append(user['id'])
            except:
                user_mentions_id = None
                
            try: 
                hash_lst = []
                for hashtag in tweet_object['entities']['hashtags']:
                    try:
                        hash_lst.append(hashtag['text'])

                    except:
                        continue

            except:
                continue

            try:
                possibly_sensitive = tweet_object['possibly_sensitive']
                
            except:
                possibly_sensitive = False

            reply_tpl = (
                tweet_object['id'],
                tweet_object['in_reply_to_status_id'],
                tweet_object['in_reply_to_user_id'],
                tweet_object['user']['id'],
                
                full_text,

                tweet_object['timestamp_ms'],
                tweet_object['lang'],
                str(hash_lst),
                tweet_object['entities']['symbols'],

                user_mentions_id,
                
                tweet_object['quote_count'],
                tweet_object['reply_count'],
                tweet_object['favorite_count'],
                tweet_object['retweet_count'],
                possibly_sensitive
            )

            replies_fill_values.append(reply_tpl)

    except:
        continue

print(replies_fill_values)

[(1131172867985485824, 1131032916232826881, 394376606, 394376606, '@British_Airways', '1558527602560', 'und', '[]', [], [18332190], 0, 0, 0, 0, False), (1131173038194606081, 1131101346801836032, 38676903, 19610469, '@easyJet It is impossible to get in contact with you by using that e-mail address. So there is no claim yet which I can follow on that damage luggage website. So what to do?', '1558527643141', 'en', '[]', [], [38676903], 0, 0, 0, 0, False), (1131173100454854657, 1131063465626427392, 43775786, 415065617, '@stoolpresidente @AmericanAir @RiggsBarstool https://t.co/b4uHb3RQA4', '1558527657985', 'und', '[]', [], [43775786, 22536055, 41091439], 0, 0, 0, 0, False), (1131173142414712832, 1131166003373723648, 210981687, 210981687, 'And had to pay extra £50 because bag wouldnt fit as carry on - at least @easyJet is bigger but shame flight was cancelled yesterday', '1558527667989', 'en', '[]', [], [38676903], 0, 0, 0, 0, False), (1131173207858470913, 1130838586281156611, 399986728, 28

## Get the quotes tweet objects

In [9]:
quotes_fill_query = """
    INSERT INTO 'replies' ( 'tweet_id', 
                            'quoted_status_id', 
                            'quoted_status_user_id', 
                            'user_id', 
                            'full_text', 
                            'timestamp_ms', 
                            'lang', 
                            'hashtags',
                            'symbols',
                            'user_mentions',
                            'quote_count',
                            'reply_count',
                            'favorite_count',
                            'retweeted_count',
                            'possibly_sensetive')
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

quotes_fill_values = []

for tweet_object in lst_main:
    try:

        if tweet_object['is_quote_status'] == True and tweet_object['text'][0:2] != 'RT':
            try: 
                if tweet_object['truncated'] == True:
                    full_text = tweet_object['extended_tweet']['full_text']

                if tweet_object['truncated'] == False:
                    full_text = tweet_object['text']
            except:
                full_text = None

            user_mentions = tweet_object['entities']['user_mentions']

            try:
                user_mentions_id = []
                for user in user_mentions:
                    user_mentions_id.append(user['id'])

            except:
                user_mentions_id = None

            try: 
                hash_lst = []
                for hashtag in tweet_object['entities']['hashtags']:
                    try:
                        hash_lst.append(hashtag['text'])

                    except:
                        continue

            except:
                continue
            
            try:
                possibly_sensitive = tweet_object['possibly_sensitive']
                
            except:
                possibly_sensitive = False


            quote_tpl = (
                tweet_object['id'],
                tweet_object['quoted_status_id'],
                tweet_object['quoted_status']['user']['id'],
                tweet_object['user']['id'],

                full_text,

                tweet_object['timestamp_ms'],
                tweet_object['lang'],
                str(hash_lst),
                tweet_object['entities']['symbols'],

                str(user_mentions_id),

                tweet_object['quote_count'],
                tweet_object['reply_count'],
                tweet_object['favorite_count'],
                tweet_object['retweet_count'],
                possibly_sensitive
            )

            quotes_fill_values.append(quote_tpl)
            
    except:
        continue

print(quotes_fill_values)

[(1131173313982734336, 1131118866082480129, 30231490, 319360391, 'Me either !!!!! Everytime I try @AmericanAir delays it 4hours + 😐😢😢', '1558527708894', 'en', '[]', [], '[22536055]', 0, 0, 0, 0, False), (1131173401895350273, 1131164515163410432, 110478967, 25155337, 'Para nada... I love @Delta', '1558527729854', 'tl', '[]', [], '[5920532]', 0, 0, 0, 0, False), (1131173644271587328, 1131030279278063616, 227687574, 307863635, 'Yes please @Ryanair @easyJet', '1558527787641', 'en', '[]', [], '[1542862735, 38676903]', 0, 0, 0, 0, False), (1131174271898836993, 1131157319486705664, 2583868590, 66965248, 'Come on @Everymancinema. You can do better than this...', '1558527937279', 'en', '[]', [], '[118667844]', 0, 0, 0, 0, False), (1131174735813832704, 1131130306101219328, 45621423, 274550264, 'Did you mean @jetairways ?', '1558528047885', 'en', '[]', [], '[14918367]', 0, 0, 0, 0, False), (1131175278825185280, 1131093579844268033, 42958829, 1645263127, '@AmericanAir come through boo', '155852817

## Get retweet tweet objects

In [10]:
retweets_fill_query = """
    INSERT INTO 'replies' ( 'tweet_id', 
                            'retweet_status_id', 
                            'user_id', 
                            'timestamp_ms', 
                            'hashtags',
                            'symbols',
                            'user_mentions',
                            'quote_count',
                            'reply_count',
                            'favorite_count',
                            'retweeted_count'
                            'retweeted_and_quoted'
                            'possibly_sensetive')
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

retweets_fill_values = []

for tweet_object in lst_main:
    try: 
        try:
            user_mentions_id = []
            for user in user_mentions:
                user_mentions_id.append(user['id'])

        except:
            user_mentions_id = None

        try:
            a = tweet_object['retweeted_status']['id']
            b = tweet_object['quoted_status']['id']
            retweeted_and_quoted = True

        except:
            retweeted_and_quoted = False

        try: 
            hash_lst = []
            for hashtag in tweet_object['entities']['hashtags']:
                try:
                    hash_lst.append(hashtag['text'])

                except:
                    continue

        except:
            continue

        try:     
            possibly_sensitive = tweet_object['possibly_sensitive']
                
        except:
            possibly_sensitive = False


        retweet_tpl = (
            tweet_object['id'],
            tweet_object['retweeted_status']['id'],
            tweet_object['user']['id'],
            tweet_object['timestamp_ms'],

            str(hash_lst),

            tweet_object['entities']['symbols'],
            
            user_mentions_id,

            tweet_object['quote_count'],
            tweet_object['reply_count'],
            tweet_object['favorite_count'],
            tweet_object['retweet_count'],
            
            retweeted_and_quoted,
            possibly_sensitive
        )

        retweets_fill_values.append(retweet_tpl)

    except:
        continue


print(retweets_fill_values)

[(1131172864147808257, 1130922003702177800, 3420691215, '1558527601645', '[]', [], [304626210], 0, 0, 0, 0, False, False), (1131172909463027720, 1131030279278063616, 36488556, '1558527612449', '[]', [], [304626210], 0, 0, 0, 0, False, False), (1131172975682605058, 1131170442809565190, 14193348, '1558527628237', '[]', [], [304626210], 0, 0, 0, 0, False, False), (1131173222039412736, 1131166315400577024, 2900385661, '1558527686973', '[]', [], [304626210], 0, 0, 0, 0, True, False), (1131173335545593856, 1131161597358018561, 224916583, '1558527714035', '[]', [], [304626210], 0, 0, 0, 0, False, False), (1131173426826231808, 1131171392991506432, 1024338385404682246, '1558527735798', '[]', [], [304626210], 0, 0, 0, 0, True, False), (1131173456597405704, 1131169807171235840, 30691018, '1558527742896', '[]', [], [304626210], 0, 0, 0, 0, False, False), (1131173508237545477, 1131150633740918785, 471878399, '1558527755208', '[]', [], [304626210], 0, 0, 0, 0, True, False), (1131173565213155329, 113

## Get the original tweet objects

In [11]:

original_tweets_fill_query = """INSERT INTO 'original_tweets' (
                                tweet_id, 
                                user_id, 
                                full_text, 
                                timestamp_ms, 
                                lang, 
                                user_mentions, 
                                hashtags, 
                                symbols, 
                                quote_count, 
                                reply_count, 
                                favourite_count, 
                                retweeted_count,
                                possibly_sensetive )  """

original_tweets_fill_values = []

def retrieving_full_text(dictionary):
    f_text = ''
    if dictionary['truncated'] == True:
        f_text = dictionary['extended_tweet'].get('full_text')
    else:
        f_text = dictionary['text']
    return f_text

def retrieving_user_mentions(dictionary):
    user_ment = []
    for user in dictionary['entities']['user_mentions']:
        try:
            user_ment.append(user['id'])
        except: 
            break
    return str(user_ment)

def retrieving_hashtags(dictionary):
    hash_lst = []
    for hashtag in dictionary['entities']['hashtags']:
        try:
            hash_lst.append(hashtag['text'])
        except:
            break
    return str(hash_lst)

def retrieving_symbols(dictionary):
    symb_lst = []
    for symbol in dictionary['entities']['symbols']:
        try:
            symb_lst.append(symbol['text'])
        except:
            break
    return str(symb_lst)


for tweet_object in lst_main:
    try:
        
        if (tweet_object['is_quote_status'] == False) and (tweet_object['text'][0:2] != 'RT') and (tweet_object['in_reply_to_status_id'] == None):

            try:
                possibly_sensitive = tweet_object['possibly_sensitive']
                
            except:
                possibly_sensitive = False


            tweet_id = tweet_object['id']
            user_id = tweet_object['user']['id']
            full_text = retrieving_full_text(tweet_object)
            timestamp_ms = tweet_object['timestamp_ms']
            user_mentions = retrieving_user_mentions(tweet_object)
            hashtags = retrieving_hashtags(tweet_object)
            symbols = retrieving_symbols(tweet_object)
            quote_count = tweet_object['quote_count']
            reply_count = tweet_object['reply_count']
            favorite_count = tweet_object['favorite_count']
            retweet_count = tweet_object['retweet_count']
            
            original_tweet_tpl = (tweet_id, user_id,full_text,timestamp_ms, user_mentions, hashtags, symbols, quote_count, reply_count, favorite_count, retweet_count, possibly_sensitive)
            
            original_tweets_fill_values.append(original_tweet_tpl)
        
    except:
        continue

print(original_tweets_fill_values)

[(1131172858951024641, 393374091, 'La ruta de easyJet entre Londres y Menorca transporta a más de 19.000 pasajeros en un año https://t.co/Rqy606KVna https://t.co/buWgtqYwCD', '1558527600406', '[]', '[]', '[]', 0, 0, 0, 0, False), (1131173010235375617, 1025334292166127617, 'EasyJet was on a slippery slope when it charged for skis - https://t.co/SOI36Thhbf #GoogleAlerts', '1558527636475', '[]', "['GoogleAlerts']", '[]', 0, 0, 0, 0, False), (1131173010508062721, 1662186764, 'Thanks @British_Airways I really needed the extra 2 hour delay on the flight today with everyone sat on board...', '1558527636540', '[18332190]', '[]', '[]', 0, 0, 0, 0, False), (1131173065122078720, 2730172562, 'So @AmericanAir @EWRairport lied to an old lady causing her to miss her flight #934 to LHR. She arrived at 6:05am for a flight departing at 8:30am,mum has no medication,is in a wheelchair and they just ignore the fact they caused this.Worse airline company. Zero customer service', '1558527649561', '[22536055,

## values and querys to load in the database

In [12]:
# FOR USER TABLE
users_fill_query
users_fill_values

# FOR ORIGINAL TWEETS TABLE
original_tweets_fill_query
original_tweets_fill_values

# FOR REPLIES TABLE
replies_fill_query
replies_fill_values

# FOR RETWEETS TABLE 
retweets_fill_query
retweets_fill_values

# FOR QUOTES TABLE
quotes_fill_query
quotes_fill_values

[(1131173313982734336,
  1131118866082480129,
  30231490,
  319360391,
  'Me either !!!!! Everytime I try @AmericanAir delays it 4hours + 😐😢😢',
  '1558527708894',
  'en',
  '[]',
  [],
  '[22536055]',
  0,
  0,
  0,
  0,
  False),
 (1131173401895350273,
  1131164515163410432,
  110478967,
  25155337,
  'Para nada... I love @Delta',
  '1558527729854',
  'tl',
  '[]',
  [],
  '[5920532]',
  0,
  0,
  0,
  0,
  False),
 (1131173644271587328,
  1131030279278063616,
  227687574,
  307863635,
  'Yes please @Ryanair @easyJet',
  '1558527787641',
  'en',
  '[]',
  [],
  '[1542862735, 38676903]',
  0,
  0,
  0,
  0,
  False),
 (1131174271898836993,
  1131157319486705664,
  2583868590,
  66965248,
  'Come on @Everymancinema. You can do better than this...',
  '1558527937279',
  'en',
  '[]',
  [],
  '[118667844]',
  0,
  0,
  0,
  0,
  False),
 (1131174735813832704,
  1131130306101219328,
  45621423,
  274550264,
  'Did you mean @jetairways ?',
  '1558528047885',
  'en',
  '[]',
  [],
  '[149183

## Loading the data in the database

In [13]:
# TO CHANGE THE HOST USER AND ECT. CHECK THE CONFIG FILE AND CHANGE THERE
from config import host, user_name, password, db_name
import pymysql

def mysqlconnect():
    # To connect MySQL database
    conn = pymysql.connect(
        host=host,
        user=user_name, 
        password = password,
        db=db_name,
        )
      
    cur = conn.cursor()
    cur.execute("select @@version")
    output = cur.fetchall()
    print(output)
      
    # To close the connection
    conn.close()
  
# Driver Code
if __name__ == "__main__" :
    mysqlconnect()

OperationalError: (2003, "Can't connect to MySQL server on '127.0.0.1' ([Errno 61] Connection refused)")